<a href="https://colab.research.google.com/github/cris-renee/AccountChecker/blob/master/Crystal_Morell_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import json

# Load the JSON file into a list of customers
with open('orders.json', 'r') as file:
    data = json.load(file)

print("JSON loaded successfully.")

JSON loaded successfully.


In [8]:
# Create a list to store the order summaries
order_summaries = []

# Loop through each customer, then through each of their orders
for customer in data:
    for order in customer.get('orders', []):
        order_id = order.get('order_id')
        total_amount = order.get('total')
        order_summaries.append((order_id, total_amount))

# Print the retrieved data
print("--- Order Summaries ---")
for oid, total in order_summaries:
    print(f"Order ID: {oid} | Total Amount: ${total:.2f}")

--- Order Summaries ---
Order ID: 5001 | Total Amount: $120.50
Order ID: 5002 | Total Amount: $250.00
Order ID: 5003 | Total Amount: $75.00


In [9]:
import pandas as pd

# 1. Normalize the nested 'items' while keeping parent 'customer' and 'order' data
df_flat = pd.json_normalize(
    data,
    record_path=['orders', 'items'],
    meta=[
        'customer_id',
        ['orders', 'order_id'],
        ['orders', 'total']
    ]
)

# 2. Rename the nested columns created by the 'meta' paths
df_flat = df_flat.rename(columns={
    'orders.order_id': 'order_id',
    'orders.total': 'total'
})

# 3. Select and reorder the columns as requested
df_flat = df_flat[['customer_id', 'order_id', 'product', 'quantity', 'total']]

# Display the final flat DataFrame
print("\n--- Flattened DataFrame ---")
print(df_flat.to_string(index=False))


--- Flattened DataFrame ---
customer_id order_id    product  quantity  total
        101     5001     Laptop         1  120.5
        101     5001      Mouse         1  120.5
        102     5002 Smartphone         1  250.0
        102     5002    Charger         2  250.0
        102     5003 Headphones         1   75.0


In [11]:
# Save the new flattened DataFrame as a JSON file
# 'records' orientation makes it a simple list of objects
df_flat.to_json('orders_cleaned.json', orient='records', indent=4)

print("Success! Your new file 'orders_cleaned.json' is saved.")

Success! Your new file 'orders_cleaned.json' is saved.
